In [1]:
import pandas as pd
from torch.utils import data
from torchvision import transforms
import numpy as np
import os
import PIL
import json
from sklearn.utils import shuffle
import cv2
import glob
import shutil
import random
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

In [2]:
train_image_folder = '10_HV_interdental_clahe'
traing_image_list = glob.glob(train_image_folder+'/*.PNG')
traing_image_list = [strn.replace('\\', "/") for strn in traing_image_list]
print(len(traing_image_list))
traing_image_list

342240


['10_HV_interdental_clahe/02-20-37-689_000408 102419 x_11_-10_LF.png',
 '10_HV_interdental_clahe/02-20-37-689_000408 102419 x_6_-10_L.png',
 '10_HV_interdental_clahe/02-20-37-721_000408 102419 x_11_-10_RF.png',
 '10_HV_interdental_clahe/02-20-37-721_000408 102419 x_6_-10_R.png',
 '10_HV_interdental_clahe/02-20-37-733_000408 102419 x_11_-9_LF.png',
 '10_HV_interdental_clahe/02-20-37-733_000408 102419 x_6_-9_L.png',
 '10_HV_interdental_clahe/02-20-37-749_000408 102419 x_11_-9_RF.png',
 '10_HV_interdental_clahe/02-20-37-749_000408 102419 x_6_-9_R.png',
 '10_HV_interdental_clahe/02-20-37-780_000408 102419 x_11_-8_LF.png',
 '10_HV_interdental_clahe/02-20-37-780_000408 102419 x_6_-8_L.png',
 '10_HV_interdental_clahe/02-20-37-796_000408 102419 x_11_-8_RF.png',
 '10_HV_interdental_clahe/02-20-37-796_000408 102419 x_6_-8_R.png',
 '10_HV_interdental_clahe/02-20-37-811_000408 102419 x_11_-7_LF.png',
 '10_HV_interdental_clahe/02-20-37-811_000408 102419 x_6_-7_L.png',
 '10_HV_interdental_clahe/02-2

In [3]:
filename = 'teeth_flip_without_rotate.json'
dict_mapping = {}
for image_path in tqdm(traing_image_list):
    if image_path.split('_')[-2] != '0':
        continue
    dict_mapping[image_path] = image_path.split('_')[-3]
dict_mapping

C:\Users\lab118\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


{'10_HV_interdental_clahe/02-20-38-065_000408 102419 x_11_0_LF.png': '11',
 '10_HV_interdental_clahe/02-20-38-065_000408 102419 x_6_0_L.png': '6',
 '10_HV_interdental_clahe/02-20-38-081_000408 102419 x_11_0_RF.png': '11',
 '10_HV_interdental_clahe/02-20-38-081_000408 102419 x_6_0_R.png': '6',
 '10_HV_interdental_clahe/02-20-38-729_000408 102419 x_10_0_LF.png': '10',
 '10_HV_interdental_clahe/02-20-38-729_000408 102419 x_7_0_L.png': '7',
 '10_HV_interdental_clahe/02-20-38-736_000408 102419 x_10_0_RF.png': '10',
 '10_HV_interdental_clahe/02-20-38-736_000408 102419 x_7_0_R.png': '7',
 '10_HV_interdental_clahe/02-20-39-753_000408 102419 x_24_0_LF.png': '24',
 '10_HV_interdental_clahe/02-20-39-753_000408 102419 x_25_0_L.png': '25',
 '10_HV_interdental_clahe/02-20-39-769_000408 102419 x_24_0_RF.png': '24',
 '10_HV_interdental_clahe/02-20-39-769_000408 102419 x_25_0_R.png': '25',
 '10_HV_interdental_clahe/02-20-40-439_000408 102419 x_23_0_LF.png': '23',
 '10_HV_interdental_clahe/02-20-40-439_

In [4]:
print(len(dict_mapping))
with open(filename, 'w+') as f:
    json.dump(dict_mapping,f)

17112


# From mapping json

In [20]:
filename = 'mapping.json'
truth_dict = {}
with open(filename,'r') as dicctionary:   

    class_nums = [0,1,2]
    concate_dict = pd.DataFrame()    
    df = json.loads(dicctionary.read())

    data = pd.DataFrame({key:value for key,value in df.items()if key in df},index = [0,1]).T

#     print('class 0 : {}'.format(len(data.loc[data[0]==0])))
#     print('class 1 : {}'.format(len(data.loc[data[0]==1])))
#     print('class 2 : {}'.format(len(data.loc[data[0]==2])))

    truth = data.loc[(data[0]==0)|(data[0]==1)|(data[0]==2)|(data[0]==3)|(data[1]==0)|(data[1]==1)|(data[1]==2)|(data[1]==3)]
    #truth.index = truth.index[:].str.slice(16)
    
    for idx, row in enumerate(truth.iterrows()):
        #truth_dict[row[0]] = truth.index[idx].split('_')[-3]
        if max(row[1]) == 0:
            truth_dict[row[0]] = str(0)
        elif max(row[1]) == 1:
            truth_dict[row[0]] = str(0)
        elif max(row[1]) == 2:
            truth_dict[row[0]] = str(1)
        elif max(row[1]) == 3:
            truth_dict[row[0]] = str(2)
truth_dict

{'Dental_Data/PBL/10_clahe_20200810/05-09-10-162_000408 102419 x_6_-10.png': '0',
 'Dental_Data/PBL/10_clahe_20200810/05-09-10-180_000408 102419 x_6_-9.png': '0',
 'Dental_Data/PBL/10_clahe_20200810/05-09-10-196_000408 102419 x_6_-8.png': '0',
 'Dental_Data/PBL/10_clahe_20200810/05-09-10-213_000408 102419 x_6_-7.png': '0',
 'Dental_Data/PBL/10_clahe_20200810/05-09-10-229_000408 102419 x_6_-6.png': '0',
 'Dental_Data/PBL/10_clahe_20200810/05-09-10-245_000408 102419 x_6_-5.png': '0',
 'Dental_Data/PBL/10_clahe_20200810/05-09-10-260_000408 102419 x_6_-4.png': '0',
 'Dental_Data/PBL/10_clahe_20200810/05-09-10-275_000408 102419 x_6_-3.png': '0',
 'Dental_Data/PBL/10_clahe_20200810/05-09-10-291_000408 102419 x_6_-2.png': '0',
 'Dental_Data/PBL/10_clahe_20200810/05-09-10-306_000408 102419 x_6_-1.png': '0',
 'Dental_Data/PBL/10_clahe_20200810/05-09-10-319_000408 102419 x_6_0.png': '0',
 'Dental_Data/PBL/10_clahe_20200810/05-09-10-332_000408 102419 x_6_1.png': '0',
 'Dental_Data/PBL/10_clahe_20

In [21]:
with open('mapping_forc.json', 'w+') as f:
    json.dump(truth_dict ,f)

In [30]:
filename = 'mapping_forc.json'
with open(filename,'r') as dicctionary:   

    concate_dict = pd.DataFrame()    
    df = json.loads(dicctionary.read())

    data = pd.DataFrame({key:value for key,value in df.items()if key in df},index = [0]).T
    print('data length =',len(data))

    if mode == 'train':
        class_2 = data[data[0]=='2'][:3820]
        class_1 = data[data[0]=='1'][:9040].sample(3820)
        class_0 = data[data[0]=='0'][:38760].sample(3820)
    elif mode == 'val':
        class_2 = data[data[0]=='2'][3820:5100]
        class_1 = data[data[0]=='1'][9040:12060].sample(1280)
        class_0 = data[data[0]=='0'][38760:51680].sample(1280)
    elif mode == 'test':
        class_2 = data[data[0]=='2'][5100:]
        class_1 = data[data[0]=='1'][12060:].sample(1280)
        class_0 = data[data[0]=='0'][51680:].sample(1280)
        
    concate_dict= pd.concat([concate_dict,class_2],sort=False,axis=0)
    concate_dict= pd.concat([concate_dict,class_1],sort=False,axis=0)
    concate_dict= pd.concat([concate_dict,class_0],sort=False,axis=0)
    concate_dict = shuffle(concate_dict)
    concate_dict[0] = concate_dict[0].astype(int)

data length = 86060


In [31]:
len(concate_dict)

11460

In [24]:
data

,0,1
Dental_Data/PBL/10_clahe_20200810/05-09-10-162_000408 102419 x_6_-10.png,0,0
Dental_Data/PBL/10_clahe_20200810/05-09-10-180_000408 102419 x_6_-9.png,0,0
Dental_Data/PBL/10_clahe_20200810/05-09-10-196_000408 102419 x_6_-8.png,0,0
Dental_Data/PBL/10_clahe_20200810/05-09-10-213_000408 102419 x_6_-7.png,0,0
Dental_Data/PBL/10_clahe_20200810/05-09-10-229_000408 102419 x_6_-6.png,0,0
...,...,...
Dental_Data/PBL/10_clahe_20200810/05-44-15-254_710456 111119 x_7_5.png,1,1
Dental_Data/PBL/10_clahe_20200810/05-44-15-278_710456 111119 x_7_6.png,1,1
Dental_Data/PBL/10_clahe_20200810/05-44-15-301_710456 111119 x_7_7.png,1,1
Dental_Data/PBL/10_clahe_20200810/05-44-15-326_710456 111119 x_7_8.png,1,1


In [19]:
max(row[1])

2

In [16]:
max(row[1],row[2])

IndexError: tuple index out of range

In [10]:
truth

,0,1
Dental_Data/PBL/10_clahe_20200810/05-09-10-162_000408 102419 x_6_-10.png,1,1
Dental_Data/PBL/10_clahe_20200810/05-09-10-180_000408 102419 x_6_-9.png,1,1
Dental_Data/PBL/10_clahe_20200810/05-09-10-196_000408 102419 x_6_-8.png,1,1
Dental_Data/PBL/10_clahe_20200810/05-09-10-213_000408 102419 x_6_-7.png,1,1
Dental_Data/PBL/10_clahe_20200810/05-09-10-229_000408 102419 x_6_-6.png,1,1
...,...,...
Dental_Data/PBL/10_clahe_20200810/05-44-15-254_710456 111119 x_7_5.png,2,2
Dental_Data/PBL/10_clahe_20200810/05-44-15-278_710456 111119 x_7_6.png,2,2
Dental_Data/PBL/10_clahe_20200810/05-44-15-301_710456 111119 x_7_7.png,2,2
Dental_Data/PBL/10_clahe_20200810/05-44-15-326_710456 111119 x_7_8.png,2,2


In [23]:
print(len(data[data[0]=='0']),len(data[data[0]=='1']),len(data[data[0]=='2']))

64600 15080 6380
